In [ ]:
!aws s3 sync s3://pumbatrainingdata/cars_detector/dataset/with_attributes/v5_0/ /home/ubuntu/mmdetection/data/v5_0 --quiet

In [1]:
%matplotlib inline
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import os
import getpass
import shutil
import json
import os
import wandb

import mmcv
from mmcv import Config
from mmdet import datasets
import mmdet.apis
from mmdet.apis import init_random_seed, set_random_seed, train_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.utils import (collect_env, get_device, get_root_logger,
                         replace_cfg_vals, setup_multi_processes,
                         update_data_root)

/opt/conda/envs/open-mmlab/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [2]:
CONFIG_NAME = 'retinanet_r50_fpn_mstrain_640-800_3x_coco-car_detector-trial'
EXP_NAME = 'exp23'
VERSION = 'v5_0'

In [3]:
pwd

'/home/ubuntu/mmdetection/training_notebooks'

In [4]:
os.chdir(f"/home/{getpass.getuser()}/mmdetection")
!mv /home/{getpass.getuser()}/mmdetection/data/{VERSION}/image_set /home/{getpass.getuser()}/mmdetection/data/.

mv: cannot stat '/home/ubuntu/mmdetection/data/v5_0/image_set': No such file or directory


In [ ]:
%%time
!python tools/train-Copy1.py configs/costume/{CONFIG_NAME}.py --work-dir work_dirs/{EXP_NAME}

In [ ]:
%%time
!python tools/test.py configs/costume/{CONFIG_NAME}.py work_dirs/{EXP_NAME}/latest.pth --out work_dirs/{EXP_NAME}/results.pkl --eval bbox --show-dir work_dirs/{EXP_NAME}/results

In [ ]:
!python tools/analysis_tools/analyze_results.py \
      configs/costume/{CONFIG_NAME}.py \
      work_dirs/{EXP_NAME}/results.pkl \
      work_dirs/{EXP_NAME}/results_analysis \
      --topk 20

# Clean disk

In [ ]:
!rm -rf /home/ubuntu/.local/share/Trash/*
!rm -rf /home/ubuntu/.cache/wandb/artifacts/*

# Inference model and visualization

In [ ]:
os.system(f"aws s3 sync s3://pumbatrainingdata/binary_model/images /home/ubuntu/Vision/images --quiet")

In [ ]:
pwd

In [ ]:
import os
import sys
import cv2
import matplotlib.pyplot as plt
import glob
import re
import getpass 
import numpy as np
from random import sample
import shutil
import pandas as pd
from tqdm.notebook import tqdm
import requests


sys.path.append('Vision/DataManagement')
os.chdir(f"/home/{getpass.getuser()}")
from utils.inference.inference import MMdetDocker, RunInference
import utils.core.common_utils as cu

In [ ]:
def yolo_get_predicions(image_path):
    url_serve = "http://odworkersgpu-asg-1-1734987265.eu-central-1.elb.amazonaws.com/detect-img"
    header = {"content-type": "image"}
    response = requests.post(url=url_serve, files={"file": ("filename", open(image_path, "rb"), "image/jpeg")}, json={})
    return response.json()[0]


def plot_predictions(result, image, yolo = False):

    if yolo:
        x1, y1, x2, y2 = map(int, result[:-1])
        score = "{:.2f}".format(result[-1])
    else:
        x1, y1, x2, y2 = map(int, result['bbox'])
        score = "{:.2f}".format(result['score'])
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 2)
    image = cv2.putText(image,
                        score, 
                        (x1, y1),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.2,
                        (0, 0, 0),
                        1,
                        cv2.LINE_AA, False)
    return image

In [ ]:
test_images = glob.glob(f"/home/ubuntu/Vision/images/large_images/*.jpg")
len(test_images), test_images[:10]

In [ ]:
inference_images = sample(test_images,200)

In [ ]:
prediction_image_path = f"/home/ubuntu/Vision/animation/yolo-v1_0-4_0"
results_dict = dict()
for i, name in tqdm(enumerate(inference_images)):
    image_name = os.path.basename(name)
    raw_image_path = f"{prediction_image_path}/test/{image_name}"
    shutil.copy2(name, raw_image_path)
    raw_image = cv2.imread(raw_image_path)
    image = cv2.resize(raw_image, (320, 240), interpolation = cv2.INTER_AREA)
    cu.save_image(image, raw_image_path)

    infer1 = RunInference(url_serve='http://127.0.0.1:8080/predictions', image_path=raw_image_path, model_name='exp13')
    infer2 = RunInference(url_serve='http://127.0.0.1:8090/predictions', image_path=raw_image_path, model_name='exp20')
    
    resultsYOLO = yolo_get_predicions(raw_image_path)
    results1 = infer1.get_predicions()
    results2 = infer2.get_predicions()

    imageYOLO = image.copy()
    image1 = image.copy()
    image2 = image.copy()
    
    for resultyolo, result1, result2 in zip(resultsYOLO, results1, results2):
        if result1['score'] > 0.8:
            image1 = plot_predictions(result1, image1)
        if result2['score'] > 0.8:
            image2 = plot_predictions(result2, image2)
        if resultyolo[-1] > 0.8:
            imageYOLO = plot_predictions(resultyolo, imageYOLO, yolo = True)
            
    results_dict[i] = {
    "image": os.path.basename(raw_image_path),
    "prediction": {
        "yolo": resultyolo,
        "exp13": result1,
        "exp20": result2
    }
}
    cu.save_image(np.concatenate([imageYOLO, image1, image2], axis=1), f"{prediction_image_path}/{image_name.replace('.jpg', '_pr.jpg')}")
    # plt.figure(figsize=(20,20)); plt.imshow(np.concatenate([imageYOLO, image1, image2], axis = 1)); plt.show()
pd.DataFrame(results_dict).to_json(f"/home/ubuntu/Vision/animation/yolo-v1_0-4_0/results/results.json")

In [ ]:
cu.clean_root('/home/ubuntu')

In [ ]:
!cd /home/ubuntu/Vision/animation; zip -rq yolo-v1_0-4_0.zip yolo-v1_0-4_0

In [ ]:
!aws s3 cp /home/ubuntu/Vision/animation/yolo-v1_0-4_0.zip s3://pumbatrainingdata/cars_detector/forensics/yolo-v1_0-4_0.zip